In [87]:
import sys

sys.path.append("../..")
from typing import List, Union

from pydantic import BaseModel, Field
from rich import print

from brain.sdk import BrainClient

In [88]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
class OperationalResult(BaseModel):
    answer: str = Field(..., description="The answer to the input query")

class RuleModification(BaseModel):
    rule_number: int = Field(..., description="The rule number to modify")
    new_content: str = Field(..., description="The new content for the rule")

class FeedbackResult(BaseModel):
    feedback: str = Field(..., description="Feedback on the previous answer")
    keep_rules: List[int] = Field(..., description="Rule numbers to keep")
    modify_rules: List[RuleModification] = Field(default_factory=list, description="Rules to modify")
    new_rules: List[str] = Field(default_factory=list, description="New rules to add")



In [4]:
from datetime import datetime
from pathlib import Path
from typing import Dict
import json

class Rule:
    def __init__(self, content: str, rule_id: int):
        self.content = content
        self.rule_id = rule_id
        self.created_at = datetime.now()
        self.last_updated = datetime.now()

class Memory:
    def __init__(self, file_path: str = "memory.json"):
        self.file_path = Path(file_path)
        self.rules: Dict[int, Rule] = {}
        self.next_rule_id = 1
        self._load()

    def _load(self):
        if not self.file_path.exists():
            self._save()
            return

        data = json.loads(self.file_path.read_text())
        self.next_rule_id = data['next_rule_id']
        self.rules = {
            int(rule_id): Rule(
                content=rule['content'],
                rule_id=int(rule_id)
            )
            for rule_id, rule in data['rules'].items()
        }

    def _save(self):
        data = {
            'next_rule_id': self.next_rule_id,
            'rules': {
                str(rule_id): {
                    'content': rule.content,
                }
                for rule_id, rule in self.rules.items()
            }
        }
        self.file_path.write_text(json.dumps(data, indent=2))

    def get_rules_context(self) -> str:
        # if not self.rules:
        #     return "No existing rules."
        
        return "Current rules:\n" + "\n".join(
            f"{rule.rule_id}. {rule.content}"
            for rule in sorted(self.rules.values(), key=lambda x: x.rule_id)
        )

    def update_from_feedback(self, feedback: FeedbackResult):
        # Convert current rules to set for easier processing
        current_rule_ids = set(self.rules.keys())
        keep_rules = set(feedback.keep_rules)
        
        # Remove rules not in keep_rules
        rules_to_remove = current_rule_ids - keep_rules
        if rules_to_remove:
            for rule_id in rules_to_remove:
                self.rules.pop(rule_id, None)

        # Modify existing rules
        if feedback.modify_rules:
            for modification in feedback.modify_rules:
                if modification.rule_number in self.rules:
                    self.rules[modification.rule_number].content = modification.new_content
                    self.rules[modification.rule_number].last_updated = datetime.now()

        # Add new rules
        if feedback.new_rules:
            for content in feedback.new_rules:
                rule = Rule(content=content, rule_id=self.next_rule_id)
                self.rules[self.next_rule_id] = rule
                self.next_rule_id += 1

        self._save()

In [ ]:
from typing import Optional, Tuple
@brain_client.reasoner(schema=OperationalResult)
def operational_reasoner(input_query: str, context: str):
    system_prompt = """You are an agent designed to provide answers based on learned rules.
Apply the rules wisely - not every rule needs to be used for every answer.
You are an agent that uses knowledge to generate accurate, generalized answers. 
Focus on principles and patterns, not specific memorization. Use the context provided to:
1. Identify underlying patterns.
2. Apply generalized knowledge to new inputs.
"""

    user_prompt = f"""Query: {input_query}

{context}

Using these rules as guidelines, provide your answer:"""

    return system_prompt, user_prompt

In [ ]:
@brain_client.reasoner(schema=FeedbackResult)
def feedback_reasoner(main_goal: str, input_query: str, answer: str, correct_answer: str, context: str):
    system_prompt = f"""You are a learning feedback agent focused on extracting deep insights aligned with a specific goal.

MAIN GOAL: {main_goal}

Your purpose is to help the system learn HOW to achieve this goal by:
1. Analyzing differences between given answers and correct answers
2. Extracting detailed knowledge that helps achieve the main goal
3. Creating comprehensive rules that guide future responses
4. Building a knowledge base focused on the goal's requirements

Core Analysis Principles:
- Extract general patterns, not specific solutions
- Look for underlying principles that achieve the goal
- Focus on knowledge that transfers to new situations
- Identify what knowledge would help achieve better answers
- Consider what deep understanding is missing
- Think about what fundamental concepts would improve performance

Rules/Knowledge Requirements:
- Must directly relate to achieving the main goal
- Should be detailed and comprehensive
- Must be generally applicable
- Should capture underlying principles
- Must focus on transferable knowledge
- Should build on existing understanding"""

    user_prompt = f"""DEEP LEARNING ANALYSIS:

GOAL TO ACHIEVE: {main_goal}

Current Learning Instance:
Input: {input_query}
Current Output: {answer}
Expected Output: {correct_answer}

Existing Knowledge Base:
{context}

Perform Deep Analysis:
1. What fundamental understanding would help achieve the goal here?
2. What deeper patterns reveal themselves when comparing outputs?
3. What core principles would improve goal achievement?
4. What essential knowledge is missing from our current rules?
5. What broader understanding would help with similar goals?

Provide:
1. Rich Analysis: Detailed feedback about what we can learn regarding our goal
2. Retained Knowledge: List numbers of rules that contain valid, goal-relevant principles
3. Knowledge Updates: List [[rule_number, "enhanced knowledge or principle"]]
4. New Understanding: Add detailed new rules that capture essential goal-relevant knowledge

Focus Areas:
- Deep understanding over surface patterns
- Core principles over specific solutions
- Transferable knowledge over contextual details
- Fundamental concepts over specific applications
- Goal achievement mechanisms over input-output pairs
Your goal is to help the system learn to achieve its main goal by analyzing successes and failures.
- Identify positive patterns that led to correct answers.
- Identify negative patterns or gaps that caused failures.
- Provide insights to refine knowledge for future queries.
- Focus on creating rules or principles that generalize well.


- If something is working give positive feedback so that the system can learn from it
- If something is not working give negative feedback so that the system can learn from it

"""

    return system_prompt, user_prompt

In [7]:
memory = Memory()
operational_reasoner_id = operational_reasoner.register()
feedback_reasoner_id = feedback_reasoner.register()

In [13]:
def process_query(input_query: str, correct_answer: Optional[str] = None, 
                 main_goal: str = "Learn to provide accurate and consistent responses"):
    memory = Memory()
    context = memory.get_rules_context()
    # Get answer using current rules
    result = brain_client.use(operational_reasoner_id)(
        input_query=input_query,
        context=context
    )

    if correct_answer is not None:
        # Get feedback and evolve rules
        feedback = brain_client.use(feedback_reasoner_id)(
            main_goal=main_goal,
            input_query=input_query,
            answer=result.answer,
            correct_answer=correct_answer,
            context=context
        )
        # Update memory based on feedback
        memory.update_from_feedback(feedback)
        
        return {"answer": result.answer, "feedback": feedback}
    
    return {"answer": result.answer}

In [9]:
# query1 = "What is the capital of France?"
# result1 = process_query(
#     input_query=query1,
#     correct_answer="Paris.",
#     main_goal="Learn how to format the answer exactly in given style"
# )
# print(result1)

In [14]:
training_examples = [
    {
        "query": "Transform 'hello'",
        "correct_answer": "Ellohay"
    },
    {
        "query": "Transform 'world'",
        "correct_answer": "Orldway"
    },
    {
        "query": "Transform 'python'",
        "correct_answer": "Ythonpay"
    },
    {
        "query": "Transform 'code'",
        "correct_answer": "Odecay"
    },
    {
        "query": "Transform 'learn'",
        "correct_answer": "Earnlay"
    }
]

def train_pattern_learner():
    memory = Memory()  # Reset memory for fresh learning
    
    for _ in range(5):
        for i, example in enumerate(training_examples, 1):
            
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation pattern that converts the input to output"
            )
            
            print(f"System Answer: {result['answer']} | Correct Answer: {example['correct_answer']}")
        print("-" * 50)

train_pattern_learner()

System Answer: eholl | Correct Answer: Ellohay

System Answer: Orldway | Correct Answer: Orldway

System Answer: Pythonay | Correct Answer: Ythonpay

System Answer: coda | Correct Answer: Odecay

System Answer: learned | Correct Answer: Earnlay

...

System Answer: orldway | Correct Answer: Orldway

System Answer: onpythay | Correct Answer: Ythonpay

System Answer: odecay | Correct Answer: Odecay

System Answer: earnlay | Correct Answer: Earnlay

--------------------------------------------------

## Enhanced multi memory CLU

```mermaid
graph TD
    A[Input Query] --> B[Operational Reasoner]
    B --> C[Generated Answer]
    C --> D[Feedback Reasoner]
    E[Correct Answer] --> D
    F[Main Goal] --> D
    
    subgraph Memory System
        G[Pattern Memory]
        H[Success Memory]
        I[Failure Memory]
        J[Confidence Tracker]
    end
    
    G --> B
    H --> B
    I --> B
    J --> B
    
    D --> K[Memory Manager]
    K --> G
    K --> H
    K --> I
    K --> J

    subgraph Feedback Analysis
        L[Pattern Extractor]
        M[Success Analyzer]
        N[Failure Analyzer]
        O[Confidence Updater]
    end
    
    D --> L
    D --> M
    D --> N
    D --> O
```

In [109]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict
from datetime import datetime
from pathlib import Path
import json

# Core Models
class KnowledgeItem(BaseModel):
    item_id: int = Field(..., description="Unique identifier for the knowledge item")
    description: str = Field(..., description="Description of what was learned")
    knowledge_type: str = Field(..., description="Type of knowledge: 'positive' for what works, 'negative' for what doesn't")
    successful_uses: int = Field(0, description="Number of times this knowledge item helped achieve correct output")
    failed_uses: int = Field(0, description="Number of times this knowledge item led to incorrect output")
    last_validated: datetime = Field(..., description="When this knowledge was last validated")
    is_general: bool = Field(..., description="Whether this is a general pattern vs specific case")

class KnowledgeUpdate(BaseModel):
    items_to_keep: List[int] = Field(..., description="IDs of knowledge items to retain")
    items_to_modify: List[str] = Field(..., description="List of [item_id, new_description] pairs")
    new_positive_items: List[str] = Field(..., description="New things that worked to be added")
    new_negative_items: List[str] = Field(..., description="New things that didn't work to be added")

class OperationalResult(BaseModel):
    answer: str = Field(..., description="The generated answer")
    applied_knowledge: List[int] = Field(..., description="IDs of knowledge items applied")
    reasoning: str = Field(..., description="Explanation of how knowledge was applied")

class FeedbackResult(BaseModel):
    success: bool = Field(..., description="Whether the answer was correct")
    analysis: str = Field(..., description="Detailed analysis of what worked/didn't work")
    knowledge_updates: KnowledgeUpdate = Field(..., description="Updates to knowledge base")


In [110]:
class Memory:
    def __init__(self, file_path: str = "memory.json", max_memory_age_days: int = 30):
        self.file_path = Path(file_path)
        self.knowledge: Dict[int, KnowledgeItem] = {}
        self.next_id = 1
        self.max_memory_age = max_memory_age_days
        self._load()

    def _load(self):
        if not self.file_path.exists():
            self._save()
            return

        data = json.loads(self.file_path.read_text())
        self.next_id = data['next_id']
        self.knowledge = {
            int(kid): KnowledgeItem(
                item_id=int(kid),
                description=k['description'],
                knowledge_type=k['knowledge_type'],
                successful_uses=k['successful_uses'],
                failed_uses=k['failed_uses'],
                last_validated=datetime.fromisoformat(k['last_validated']),
                is_general=k['is_general']
            )
            for kid, k in data.get('knowledge', {}).items()
        }

    def _save(self):
        data = {
            'next_id': self.next_id,
            'knowledge': {
                str(k.item_id): {
                    'description': k.description,
                    'knowledge_type': k.knowledge_type,
                    'successful_uses': k.successful_uses,
                    'failed_uses': k.failed_uses,
                    'last_validated': k.last_validated.isoformat(),
                    'is_general': k.is_general
                }
                for k in self.knowledge.values()
            }
        }
        self.file_path.write_text(json.dumps(data, indent=2))

    def get_context(self) -> str:
        current_time = datetime.now()
        
        positive_knowledge = "\nWhat Works (Positive Knowledge):\n" + "\n".join(
            f"K{k.item_id}. {k.description} (successes: {k.successful_uses}, failures: {k.failed_uses})"
            for k in self.knowledge.values()
            if k.knowledge_type == 'positive' 
            and (current_time - k.last_validated).days <= self.max_memory_age
        )

        negative_knowledge = "\nWhat Doesn't Work (Negative Knowledge):\n" + "\n".join(
            f"K{k.item_id}. {k.description} (observed {k.failed_uses} times)"
            for k in self.knowledge.values()
            if k.knowledge_type == 'negative'
            and (current_time - k.last_validated).days <= self.max_memory_age
        )

        return f"{positive_knowledge}\n{negative_knowledge}"

    def update_from_feedback(self, feedback: FeedbackResult):
        current_time = datetime.now()
        updates = feedback.knowledge_updates
        
        # Remove items not in keep list
        for item_id in set(self.knowledge.keys()) - set(updates.items_to_keep):
            self.knowledge.pop(item_id, None)
        
        # Update successful/failed uses based on feedback
        for item_id in updates.items_to_keep:
            if item_id in self.knowledge:
                item = self.knowledge[item_id]
                if feedback.success:
                    item.successful_uses += 1
                else:
                    item.failed_uses += 1
                item.last_validated = current_time
        
        # Add new positive knowledge
        for description in updates.new_positive_items:
            item = KnowledgeItem(
                item_id=self.next_id,
                description=description,
                knowledge_type='positive',
                successful_uses=1 if feedback.success else 0,
                failed_uses=0 if feedback.success else 1,
                last_validated=current_time,
                is_general=True  # Assuming new knowledge is general unless specified
            )
            self.knowledge[self.next_id] = item
            self.next_id += 1

        # Add new negative knowledge
        for description in updates.new_negative_items:
            item = KnowledgeItem(
                item_id=self.next_id,
                description=description,
                knowledge_type='negative',
                successful_uses=0,
                failed_uses=1,
                last_validated=current_time,
                is_general=True
            )
            self.knowledge[self.next_id] = item
            self.next_id += 1

        self._save()

In [111]:
@brain_client.reasoner(schema=OperationalResult)
def operational_reasoner(input_query: str, context: str):
    system_prompt = """You are an agent that learns from experience what works and what doesn't.
Your goal is to apply successful patterns while avoiding known pitfalls.

When generating a response:
1. Review what has worked in the past (positive knowledge)
2. Consider what hasn't worked (negative knowledge)
3. Apply general patterns rather than specific cases
4. Explain your reasoning process"""

    user_prompt = f"""Query: {input_query}

Available Knowledge:
{context}

Provide:
1. Your answer 
2. List of knowledge IDs (K#) you applied
3. Explanation of how you used this knowledge

Guidelines:
- Focus on applying general patterns that have succeeded
- Avoid approaches that have failed
- Explain why you chose certain knowledge items
- Don't just pattern match against examples"""

    return system_prompt, user_prompt

@brain_client.reasoner(schema=FeedbackResult)
def feedback_reasoner(main_goal: str, input_query: str, answer: str, correct_answer: str, context: str):
    system_prompt = f"""You are a reinforcement learning inspired agent that identifies what works and what doesn't.

MAIN GOAL: {main_goal}

Your job is to:
1. Determine if the answer was successful
2. Identify what worked (positive reinforcement)
3. Identify what didn't work (negative reinforcement)
4. Extract general patterns, not specific examples
5. Build knowledge that helps achieve the goal

Key Principles:
- Focus on general patterns over specific cases
- Look for underlying transformation rules
- Track both successful and failed approaches
- Build transferable knowledge"""

    user_prompt = f"""Analysis Task:
Analyze what worked and what didn't in this example.

Current Case:
Input: {input_query}
System Output: {answer}
Expected Output: {correct_answer}

Current Knowledge:
{context}

Provide:
1. success: Was the answer correct? (true/false)
2. analysis: Detailed analysis of what worked/didn't work
3. knowledge_updates:
   - items_to_keep: List of knowledge IDs (K#) that remain valid
   - items_to_modify: List of [knowledge_id, new_description] pairs
   - new_positive_items: List of new things that worked
   - new_negative_items: List of new things that didn't work

Remember:
- Focus on general transformation patterns
- Don't memorize specific examples
- Identify what makes solutions work or fail
- Build knowledge that can apply to new cases"""

    return system_prompt, user_prompt


#register the reasoners
operational_reasoner_id = operational_reasoner.register()
feedback_reasoner_id = feedback_reasoner.register()

In [112]:
def process_query(input_query: str, correct_answer: Optional[str] = None, 
                 main_goal: str = "Learn to provide accurate and consistent responses",
                 memory_file: str = "memory.json",
                 max_memory_age_days: int = 30):
    
    memory = Memory(file_path=memory_file, max_memory_age_days=max_memory_age_days)
    context = memory.get_context()
    
    # Get answer using current knowledge
    result = brain_client.use(operational_reasoner_id)(
        input_query=input_query,
        context=context
    )

    if correct_answer is not None:
        # Get feedback and evolve knowledge
        feedback = brain_client.use(feedback_reasoner_id)(
            main_goal=main_goal,
            input_query=input_query,
            answer=result.answer,
            correct_answer=correct_answer,
            context=context
        )
        
        # Update memory based on feedback
        memory.update_from_feedback(feedback)
        
        return {"answer": result.answer, "feedback": feedback}
    
    return {"answer": result.answer}

In [ ]:
training_examples = [
    {
        "query": "Transform 'hello'",
        "correct_answer": "Ellohay"
    },
    {
        "query": "Transform 'world'",
        "correct_answer": "Orldway"
    },
    {
        "query": "Transform 'python'",
        "correct_answer": "Ythonpay"
    },
    {
        "query": "Transform 'code'",
        "correct_answer": "Odecay"
    },
    {
        "query": "Transform 'learn'",
        "correct_answer": "Earnlay"
    }
]

def train_pattern_learner():
    for _ in range(5):
        for i, example in enumerate(training_examples, 1):
            
            result = process_query(
                input_query=example['query'],
                correct_answer=example['correct_answer'],
                main_goal="Learn the hidden transformation pattern that converts the input to output, and return the correct output",
                memory_file = "test_2.json",
                max_memory_age_days = 30
            )
            
            print(f"System Answer: {result['answer']} | Correct Answer: {example['correct_answer']}")
        print("-" * 50)

train_pattern_learner()

System Answer: olleh | Correct Answer: Ellohay

System Answer: orldway | Correct Answer: Orldway

System Answer: ythonpay | Correct Answer: Ythonpay

System Answer: odeway | Correct Answer: Odecay

System Answer: earnlay | Correct Answer: Earnlay

...

System Answer: orldway | Correct Answer: Orldway

System Answer: ythonpay | Correct Answer: Ythonpay

System Answer: odecay | Correct Answer: Odecay

System Answer: earnlay | Correct Answer: Earnlay

--------------------------------------------------